In [640]:
# importing the needed libraries/packages/modules

import numpy as np
import pandas as pd
import math
from graphviz import Digraph

from sklearn import datasets
from sklearn.model_selection import train_test_split

In [641]:
# getting the iris dataset and initializing the input, output and features

data=datasets.load_iris()

X=data.data
Y=data.target
features=data.feature_names

In [642]:
# splitting the data in train and test

X_train, X_test, Y_train, Y_test=train_test_split(X, Y, random_state=1)

In [643]:
# creating a graph object

dot=Digraph(comment="Decision Tree")

In [644]:
# tree node class for holding the data of a desicion tree node 

class TreeNode : 
    def __init__(self) :
        # since all the features have continuous values therefore, there will be only two splits wrt a feature(if the split happens) and i.e. why we're maintaing left and right pointers only

        self.left=None
        self.right=None        

In [645]:
# get gain ratio function which returns the gain ratio after splitting wrt a particular feature and and split value 

def getGainRatio(X, Y, feature_idx, split_value, info_gain_before_splitting) :    
    values_left, counts_left=np.unique(Y[np.where(X[:, feature_idx]<=split_value)], return_counts=True)
    values_right, counts_right=np.unique(Y[np.where(X[:, feature_idx]>split_value)], return_counts=True)

    # getting the information gain for the left and right nodes formed after splitting

    info_gain_left=getEntropy(counts_left)
    info_gain_right=getEntropy(counts_right)

    sum_left=sum(counts_left)
    sum_right=sum(counts_right)        

    # getting the net information gain after the split

    net_info_gain_after_splitting=(((sum_left*info_gain_left)+(sum_right*info_gain_right))/Y.shape[0])    
    split_info=-(((sum_left*math.log2(sum_left/Y.shape[0]))+(sum_right*math.log2(sum_right/Y.shape[0])))/Y.shape[0])           

    #  getting the gain ratio wrt the given feature and split value 

    gain_ratio=((info_gain_before_splitting- net_info_gain_after_splitting)/split_info)

    return gain_ratio

In [646]:
# get entropy function which returns the entropy for the division of data points wrt different features at a particular node 

def getEntropy(counts) :
    # the entropy for a pure node is 0

    if(len(counts)==1) :
        return 0
        
    # getting the entropy for the node with the division of datapoints as indicated by counts

    total_counts=sum(counts)

    prob=(counts/total_counts)    
    log_prob=[math.log2(i) for i in prob]
    
    entropy=-sum(prob*log_prob)

    return entropy

In [647]:
# print decision tree function which prints the decision tree in a depth first manner wrt the training data, makes the tree and the graph too

def printDT(X, Y, features, curr_level, curr_node, prev_node, edges):
    # getting the division of data points at the current node

    values, counts=np.unique(Y, return_counts=True)    

    # printing the level, each feature and its count and the current entropy 

    print("Level", curr_level)    

    # initializing the graph node's name and text

    graph_node_name=str(curr_node[0])
    graph_node_text=""            

    for i in range(len(values)) :
        print("Count of", values[i], "=", counts[i])

        # adding frequencies to node text
        
        graph_node_text+=str(data.target_names[values[i]])
        graph_node_text+=" = "
        graph_node_text+=str(counts[i])
        graph_node_text+='\n'

    curr_entropy=getEntropy(counts)    

    # adding entropy to node text

    graph_node_text+="entropy = "
    graph_node_text+="{:.3f}".format(curr_entropy)
    graph_node_text+='\n'    

    print("Current Entropy is =", "{:.3f}".format(curr_entropy))

    # returning a leaf node with its prediction if we have a pure node or no more features to split upon(indicated by all -1 in features)

    if len(values)==1 :        
        root=TreeNode()
        root.prediction=values[0]

        # adding predictions to node text, making a corresponding graph node, adding the edge to the edges array and incrementing the current node
        
        graph_node_text+=str(data.target_names[root.prediction])
        graph_node_text+='\n'    

        dot.node(graph_node_name, graph_node_text)
        edges.append(str(prev_node)+str(curr_node[0]))
        
        curr_node[0]=chr(ord(curr_node[0])+1)

        print("Reached leaf Node", end='\n\n')

        return root

    if len(set(features))==1 :
        root=TreeNode()
        root.prediction=values[np.argmax(counts)]

        # adding predictions to node text, making a corresponding graph node, adding the edge to the edges array and incrementing the current node

        graph_node_text+=str(data.target_names[root.prediction])
        graph_node_text+='\n'    
        
        dot.node(graph_node_name, graph_node_text)
        edges.append(str(prev_node)+str(curr_node[0]))  
        
        curr_node[0]=chr(ord(curr_node[0])+1)

        print("Reached leaf Node", end='\n\n')

        return root

    # getting the max gain ratio, the corresponding feature, its index and the split value

    max_gain_ratio=None
    max_gain_ratio_feature=None    
    max_idx=None
    split_value=None    

    # iterating on all the features available for split(indicated as non -1)

    for j in range(len(features)) :
        if features[j]==-1 :
            continue

        # getting the unique sorted input values wrt the current feature

        X_sorted=np.unique(X[:, j])

        # iterating on all the possible mid values and finding the one which corresponds to the maximun gain ratio for the current feature 

        for i in range(len(X_sorted)-1) :            
                mid=((X_sorted[i]+X_sorted[i+1])/2)
                
                gain_ratio=getGainRatio(X, Y, j, mid, curr_entropy)

                if max_gain_ratio==None or gain_ratio>max_gain_ratio :            
                    max_gain_ratio=gain_ratio
                    max_gain_ratio_feature=features[j]
                    max_idx=j
                    split_value=mid

    # printing the feature we're splitting upon, along with the split value and gain ratio
    
    print("Splitting on feature", max_gain_ratio_feature[:-5], "(<=", "{:.3f}".format(split_value), "cm) with gain ratio", "{:.3f}".format(max_gain_ratio), end='\n\n')  

    # adding gain ratio, splitting feature and split value to node text

    graph_node_text+="gain ratio = "
    graph_node_text+="{:.3f}".format(max_gain_ratio)
    graph_node_text+='\n' 
    graph_node_text+=max_gain_ratio_feature[:-5]
    graph_node_text+=" (cm) "
    graph_node_text+="(<="
    graph_node_text+="{:.3f}".format(split_value)
    graph_node_text+=")"
    graph_node_text+='\n' 

    # making a corresponding graph node, adding the edge to the edges array(if prev node is not empty) and incrementing the setting prev node to current and incrementing the current node

    dot.node(graph_node_name, graph_node_text)

    if(prev_node!='') :
        edges.append(str(prev_node)+str(curr_node[0]))

    prev_node=curr_node[0]        
    curr_node[0]=chr(ord(curr_node[0])+1)    

    # making a tree node with the feature to split upon and the split value 

    root=TreeNode()

    root.feature_idx=max_idx
    root.split_value=split_value

    # getting the new features for the left and the right split

    new_features_left=[i for i in features]
    new_features_left[max_idx]=-1

    new_features_right=[i for i in features]
    new_features_right[max_idx]=-1

    # getting the indices of the input to be passed to the left and the right decision trees

    select_indices_left=np.where(X[:, max_idx]<=split_value)  
    select_indices_right=np.where(X[:, max_idx]>split_value)    

    # printing the left and right decision trees and getting their root nodes too    

    left=printDT(X[select_indices_left], Y[select_indices_left], new_features_left, curr_level+1, curr_node, prev_node, edges)   
    right=printDT(X[select_indices_right], Y[select_indices_right], new_features_right, curr_level+1, curr_node, prev_node, edges)    

    # attaching the left and right subtrees to the root node and then returning the root

    root.left=left
    root.right=right
    
    return root

In [648]:
# printing the decision tree for the training data, getting the root of the tree formed and rendering the tree inside Iris.gv

# initializing the edges array, previous and current node

edges=[]
prev_node=''
curr_node=['A']

root=printDT(X_train, Y_train, features, 0, curr_node, prev_node, edges)

# adding edges to the graph and rendering it

dot.edges(edges)
dot.render("Iris.gv")

Level 0
Count of 0 = 37
Count of 1 = 34
Count of 2 = 41
Current Entropy is = 1.581
Splitting on feature petal length (<= 2.600 cm) with gain ratio 1.000

Level 1
Count of 0 = 37
Current Entropy is = 0.000
Reached leaf Node

Level 1
Count of 1 = 34
Count of 2 = 41
Current Entropy is = 0.994
Splitting on feature petal width (<= 1.650 cm) with gain ratio 0.661

Level 2
Count of 1 = 33
Count of 2 = 4
Current Entropy is = 0.494
Splitting on feature sepal length (<= 7.100 cm) with gain ratio 0.511

Level 3
Count of 1 = 33
Count of 2 = 3
Current Entropy is = 0.414
Splitting on feature sepal width (<= 2.850 cm) with gain ratio 0.065

Level 4
Count of 1 = 19
Count of 2 = 3
Current Entropy is = 0.575
Reached leaf Node

Level 4
Count of 1 = 14
Current Entropy is = 0.000
Reached leaf Node

Level 3
Count of 2 = 1
Current Entropy is = 0.000
Reached leaf Node

Level 2
Count of 1 = 1
Count of 2 = 37
Current Entropy is = 0.176
Splitting on feature sepal length (<= 5.950 cm) with gain ratio 0.097

Level

'Iris.gv.pdf'

In [649]:
# predict function which predicts the class wrt the input testing data point using the decision tree

def predict(root, X) :
    # returning the prediction if we reach a leaf node

    if(root.left==None and root.right==None) :
        return root.prediction

    # moving left or right depending on the value of the specific feature the node holds(comparing it with the respective split value)

    if(X[root.feature_idx]<=root.split_value) :
        return predict(root.left, X)

    return predict(root.right, X)

In [650]:
# getting all the predictions for the testing data

Y_pred=[]

for i in X_test :
    Y_pred.append(predict(root, i))

In [651]:
# score function which returns the mean accuracy by comaring the predictions with the truth values 

def score(Y_pred, Y_true) :
    count=0

    for i in range(len(Y_pred)) :
        if(Y_pred[i]==Y_true[i]) :
            count+=1

    return (count/len(Y_pred))

In [652]:
# getting the score of the testing data on the decision tree

score(Y_pred, Y_test)

0.9736842105263158